Carrega as bibliotecas e testa a conexão com banco de dados PostGreeSQL

In [1]:
import connection as cn
from sqlalchemy import text, inspect, Engine
import pandas as pd

cn.connection_test()

✅ Sucesso: Conexão estabelecida com o banco de dados.


In [ ]:
# 1. Configurações iniciais
con = cn.get_session()
schema_name = 'analytics'
dados_tabelas = []

# 2. Busca otimizada dos nomes das tabelas e criação do inspector
# Passamos con.bind (a Engine) para inspect()
inspector = inspect(con.bind) 
tabelas = inspector.get_table_names(schema=schema_name)

# 3. Itera sobre as tabelas, obtém contagem E colunas
for tabela in tabelas:
    # 3.1. Obtém a lista de colunas
    try:
        # Usa o inspector para obter a lista de dicionários de colunas
        colunas_info = inspector.get_columns(tabela, schema=schema_name)
        
        # Extrai apenas o nome ('name') de cada coluna e cria a lista [col1, col2, ...]
        lista_colunas = [col['name'] for col in colunas_info]
        
    except Exception as e:
        # Em caso de erro ao inspecionar colunas (ex: tabela inacessível)
        lista_colunas = f"ERRO INSPECT: {e.__class__.__name__}"


    # 3.2. Obtém a contagem de registros (igual ao código original)
    try:
        sql_count = text(f"SELECT COUNT(*) FROM {schema_name}.{tabela};")
        count = con.execute(sql_count).scalar_one()

    except Exception as e:
        count = f"ERRO COUNT: {e.__class__.__name__}"
        
        
    # 3.3. Adiciona todos os dados (incluindo a nova coluna) à lista
    dados_tabelas.append({
        'tablename': tabela,
        'record_count': count,
        'columns_list': lista_colunas  # << NOVA COLUNA AQUI
    })

# 4. Cria o DataFrame e imprime
#df_tabelas = pd.DataFrame(dados_tabelas)
#df_tabelas.to_csv('tabelas_colunas.csv')
#df_tabelas

Executa consultas SQL e retorna resultado em um dataframe PANDAS

In [7]:
session = cn.get_session().bind
consulta = text("""
SELECT
    d.year,
    d.month,
    SUM(f.price + f.freight_value) AS total_vendas
FROM fact_sales f
JOIN dim_date d 
    ON f.date_sk = d.date_sk
GROUP BY 
    d.year,
    d.month
ORDER BY 
    d.year,
    d.month;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head()

#SUM(price + freight_value) → soma o valor pago pelo produto + frete.
#A tabela dim_date fornece ano e mês, permitindo o agrupamento mensal.
#GROUP BY organiza os resultados por ano e mês.


,year,month,total_vendas
0,2016,9,354.75
1,2016,10,56808.84
2,2016,12,19.62
3,2017,1,137188.49
4,2017,2,286280.62


In [ ]:
session = cn.get_session().bind
consulta = text("""
SELECT
    f.product_sk,
    COUNT(f.product_sk) AS quantidade_vendida
FROM fact_sales f
GROUP BY
    f.product_sk
ORDER BY
    quantidade_vendida DESC
LIMIT 10;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head()

#As 10 categorias com maior quantidade de vendas.
#COUNT(product_sk) conta cada venda registrada na fact table.
#Ordena do maior para o menor.

In [ ]:
session = cn.get_session().bind
consulta = text("""
SELECT
    f.seller_sk,
    f.product_sk,
    d.year,
    d.month,
    SUM(f.price + f.freight_value) AS total_vendas
FROM fact_sales f
JOIN dim_date d
    ON f.date_sk = d.date_sk
GROUP BY CUBE (
    f.seller_sk,
    f.product_sk,
    d.year,
    d.month
)
ORDER BY
    f.seller_sk NULLS LAST,
    f.product_sk NULLS LAST,
    d.year NULLS LAST,
    d.month NULLS LAST;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head()

#O GROUP BY CUBE vai gerar automaticamente:
#✔ Detalhamento máximo:
#seller_sk + product_sk + ano + mês
#✔ Totais parciais:
#seller_sk + product_sk
#seller_sk + mês
#product_sk + mês
#somente seller_sk
#somente product_sk
#somente período (year, month)
#✔ Total geral:
#todos os campos NULL

In [5]:
session = cn.get_session().bind
consulta = text("""
WITH Customer_First_Order AS (
    -- 1. Encontra a data da primeira compra e trunca para o início do mês (Coorte)
    SELECT
        t1.customer_sk,
        -- CAST para garantir que a DB trate a coluna como TIMESTAMP
        DATE_TRUNC('month', MIN(CAST(t2.order_purchase_timestamp AS TIMESTAMP))) AS cohort_start_date
    FROM
        fact_sales AS t1
    JOIN
        df_orders AS t2 ON t1.order_id = t2.order_id
    GROUP BY
        t1.customer_sk
),
Customer_Activity AS (
    -- 2. Combina o mês da coorte com o mês de cada compra subsequente e calcula a diferença em meses
    SELECT
        cfo.customer_sk,
        TO_CHAR(cfo.cohort_start_date, 'YYYY-MM') AS cohort_month, -- Mês da Primeira Compra (Coorte formatado)
        
        -- Calcula o número de meses entre o início do mês da compra atual e o início do mês da coorte.
        (
            (EXTRACT(YEAR FROM DATE_TRUNC('month', CAST(t2.order_purchase_timestamp AS TIMESTAMP))) - EXTRACT(YEAR FROM cfo.cohort_start_date)) * 12 +
            (EXTRACT(MONTH FROM DATE_TRUNC('month', CAST(t2.order_purchase_timestamp AS TIMESTAMP))) - EXTRACT(MONTH FROM cfo.cohort_start_date))
        ) AS month_number -- Número de meses desde a coorte (0 = Mês da Coorte)
    FROM
        Customer_First_Order AS cfo
    JOIN
        fact_sales AS t1 ON cfo.customer_sk = t1.customer_sk
    JOIN
        df_orders AS t2 ON t1.order_id = t2.order_id
    GROUP BY
        cfo.customer_sk,
        cfo.cohort_start_date,
        -- Agrupar pelo início do mês da compra atual (com CAST)
        DATE_TRUNC('month', CAST(t2.order_purchase_timestamp AS TIMESTAMP))
)
-- 3. Calcula o número de clientes retidos e a taxa de retenção
SELECT
    ca.cohort_month,
    ca.month_number,
    COUNT(DISTINCT ca.customer_sk) AS retained_customers,
    -- Calcula o tamanho inicial da coorte (clientes no Month 0)
    (
        SELECT
            COUNT(DISTINCT customer_sk)
        FROM
            Customer_Activity
        WHERE
            cohort_month = ca.cohort_month AND month_number = 0
    ) AS cohort_size,
    -- Calcula a taxa de retenção (Retained / Cohort Size)
    CAST(COUNT(DISTINCT ca.customer_sk) AS REAL) * 100 /
    (
        SELECT
            COUNT(DISTINCT customer_sk)
        FROM
            Customer_Activity
        WHERE
            cohort_month = ca.cohort_month AND month_number = 0
    ) AS retention_rate
FROM
    Customer_Activity AS ca
GROUP BY
    ca.cohort_month,
    ca.month_number
ORDER BY
    ca.cohort_month,
    ca.month_number;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head()

#-- Resumo da Análise de Cohort/Retenção:
#1. IDENTIFICAÇÃO DA COORTE: Define o mês da primeira compra (coorte) para cada cliente.
#2. CÁLCULO DO DESLOCAMENTO: Calcula a diferença em meses entre o mês de cada compra e o mês da coorte.
#3. CÁLCULO DA RETENÇÃO: Agrupa o resultado para contar clientes retidos e calcular a taxa de retenção ao longo do tempo.

,cohort_month,month_number,retained_customers,cohort_size,retention_rate
0,2016-09,0.0,3,3,100.0
1,2016-10,0.0,308,308,100.0
2,2016-12,0.0,1,1,100.0
3,2017-01,0.0,789,789,100.0
4,2017-02,0.0,1733,1733,100.0


In [6]:
session = cn.get_session().bind
consulta = text("""
WITH Order_Revenue AS (
    -- 1. Calcula a Receita Total (Preço + Frete) para CADA Pedido (Order_ID)
    SELECT
        t1.order_id,
        -- Conversão explícita para TIMESTAMP para evitar erro de tipo de dados
        CAST(t2.order_purchase_timestamp AS TIMESTAMP) AS order_timestamp,
        SUM(t1.price + t1.freight_value) AS total_order_revenue
    FROM
        fact_sales AS t1
    JOIN
        df_orders AS t2 ON t1.order_id = t2.order_id
    GROUP BY
        t1.order_id,
        t2.order_purchase_timestamp -- Agrupamento para somar os itens de um mesmo pedido
)
-- 2. Calcula o Ticket Médio por Mês (Receita Total / Número de Pedidos Distintos)
SELECT
    -- Trunca a data para o início do mês e formata para 'YYYY-MM'
    TO_CHAR(DATE_TRUNC('month', order_timestamp), 'YYYY-MM') AS order_month, 
    COUNT(DISTINCT order_id) AS total_orders, 
    SUM(total_order_revenue) AS total_revenue, 
    -- Ticket Médio = (Soma da Receita) / (Contagem Distinta de Pedidos)
    ROUND(CAST(SUM(total_order_revenue) AS NUMERIC) / COUNT(DISTINCT order_id), 2) AS average_ticket
FROM
    Order_Revenue
GROUP BY
    order_month
ORDER BY
    order_month;
""")
df = pd.read_sql(sql=consulta, con=session)
df.head()

#Análise: Ticket Médio Mensal
#1. AGREGAÇÃO POR PEDIDO: Soma 'price' e 'freight_value' para calcular a receita total por order_id.
#2. AGREGAÇÃO POR MÊS: Agrupa os pedidos pelo mês de compra.
#3. CÁLCULO FINAL: Divide a Receita Total pela Contagem de Pedidos Distintos (Ticket Médio).





,order_month,total_orders,total_revenue,average_ticket
0,2016-09,3,354.75,118.25
1,2016-10,308,56808.84,184.44
2,2016-12,1,19.62,19.62
3,2017-01,789,137188.49,173.88
4,2017-02,1733,286280.62,165.19
